Administrators prepare initial plan data which then becomes available to managers. One
planning period stands for one quarter. Administrators have access to all plan data unlike managers.

Managers plan sales in several countries. Mapping between countries and managers is
documented in the system’s settings. Once the administrator has prepared the initial version of sales
plan, managers will be able to update and approve the calculated figures. The initial sales plan is based
on actual data in two previous years.

Managers can lock their data to avoid unintended corrections by other users. Only locked data
is available for correction.

At the end of the planning process managers confirm correctness of plan figures. The approved
plan is then used to prepare a report on plan-fact deviations.

Plan-fact comparison report is available to both managers and administrators

```
create role administrators;

create role managers;

create user admin1 with password '123'

create user manager1 with password '123'
create user manager2 with password '123'

grant administrators to admin1;

grant managers to manager1;
grant managers to manager2;

grant usage on schema public to administrators;
grant select, update on plans to administrators;
```

---

A plan data slice is defined by a quarter (e.g., 2013.3) and a country.

Plan data has three versions: - N (initial calculation), P (edited) and A (confirmed and considered as the result of planning).

The status of each data R (ready for work), L (locked by manager and available for editing), A (approved).

• The planning period (quarter and year) is defined
• The administrator creates current planning period records with an R status for each slice of plan
data. If there are 6 countries, then 6 records will be added.
• Initial plan calculation is performed, and the results are saved as an ‘N’ version. Version N is
not changed afterwards.
• Plan data is copied from N to P version. The only difference between these versions of a plan
is only the version name
• Managers change P version of plan data after changing the respective slice status from R to L.
If the data was locked by another person before, then editing is impossible.
• Managers release locks on their data. Removing data locks means changing data status from L
to R. The lock can be removed only if the slice status is L and the author of last changes is the
current user.
• Then the data is copied to the ‘A’ version by managers which means that the plan data is
approved. The respective status records of data slices are changed from R to A. Data in the ‘A’
version is then used to prepare a report on differences between planed and actual sales amount.

```
CREATE TABLE plan_status (
	quarterid varchar(6) NOT NULL,
	status varchar(10) NOT NULL,
	modifieddatetime timestamp NOT NULL DEFAULT CURRENT_TIMESTAMP,
	author varchar(20) NOT NULL DEFAULT CURRENT_USER,
	country varchar(5) NOT NULL,
	CONSTRAINT plan_status_pk PRIMARY KEY (quarterid, country)
);
CREATE TABLE plan_data (
	versionid varchar(1) NOT NULL,
	country varchar(5) NOT NULL,
	quarterid varchar(6) NOT NULL,
	pcid int4 NOT NULL,
	salesamt numeric(18,2) NULL,
	CONSTRAINT planapp_data_pkey PRIMARY KEY (quarterid, country, pcid, versionid)
);
CREATE TABLE country_managers (
	username varchar(30) NOT NULL,
	country varchar(5) NOT NULL,
	CONSTRAINT country_managers_pk PRIMARY KEY (username, country)
);

CREATE TABLE company (
	id int4 NOT NULL GENERATED ALWAYS AS IDENTITY,
	cname varchar(200) NOT NULL,
	countrycode varchar(10) NULL,
	city varchar(30) NULL,
	CONSTRAINT d_company_pk PRIMARY KEY (id)
);
CREATE TABLE company_sales (
	cid int4 NOT NULL,
	salesamt numeric(18,2) NULL,
	year int4 NULL,
	quarter_yr int4 NULL,
	qr varchar(6) NOT NULL,
	categoryid int4 NOT NULL,
	ccls varchar(1) NULL,
	CONSTRAINT company_sales_pk PRIMARY KEY (qr, cid, categoryid)
);
CREATE TABLE company_abc (
	cid int4 NOT NULL,
	salestotal numeric NULL,
	cls varchar(1) NULL,
	year int4 NOT NULL,
	CONSTRAINT company_abc_pk PRIMARY KEY (cid, year)
);



create view v_plan_edit as
select pd.country, pd.quarterid, pd.pcid, pd.salesamt, pd.versionid
	from plan_data pd 
	where 
		pd.versionid = 'P' 
		and 
			pd.country in (select country 
				from country_managers cm 
					where cm.username = current_user)
		and 
			pd.quarterid in (select ps.quarterid 
				from plan_status ps 
					where ps.author = current_user and ps.status = 'L');



create view v_plan as
select pd.country,
    pd.pcid,
    pd.quarterid,
    pd.salesamt
   FROM plan_data pd
  WHERE pd.versionid = 'A' 
  	AND (pd.country IN (SELECT cm.country FROM country_managers cm
          WHERE cm.username = CURRENT_USER) 
          or 
          pg_has_role(current_user, 'planadmin', 'member')) 
    AND (pd.quarterid IN ( SELECT ps.quarterid
           FROM plan_status ps
          WHERE ps.status = 'A'));

create role planadmin;
create role planmanager;

```